#Подготовка

## Импотры и инсталяции необходимых модулей

In [ ]:
#Подключим Google диск
from google.colab import drive
drive.mount('/content/drive')
!cp -r ./drive/My\ Drive/FinalProject/* .

Mounted at /content/drive


In [ ]:
!pip install -q catboost shap

     |████████████████████████████████| 76.1 MB 1.2 MB/s 
     |████████████████████████████████| 564 kB 75.9 MB/s 


In [ ]:
!pip install -q gpx-cmd-tools

     |████████████████████████████████| 111 kB 36.1 MB/s 


In [ ]:
!pip install -q python-tcxparser

In [ ]:
#!pip install -q pandas-profiling==3.1.0

In [ ]:
!sudo apt install -q pv

Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'sudo apt autoremove' to remove it.
Suggested packages:
  doc-base
The following NEW packages will be installed:
  pv
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 48.3 kB of archives.
After this operation, 123 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 pv amd64 1.6.6-1 [48.3 kB]
Fetched 48.3 kB in 0s (155 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preco

In [ ]:
import catboost as ctb
from catboost import Pool, cv
from google.colab import output
from pandas_profiling import ProfileReport
from feature_selector import FeatureSelector
import numpy as np
import pandas as pd
import seaborn as sns
import shap
import os
import gzip
import gc
import folium
import zipfile
from matplotlib import pyplot 
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.inspection import permutation_importance
from sklearn import preprocessing
import matplotlib.pyplot as plt

## Загрузка данных

In [ ]:
#Разархивируем
!unzip all_data.zip | pv -l >/dev/null
!unzip activities.zip | pv -l >/dev/null

1.00  0:00:11 [0.00 /s] [ <=>                                                  ]
2.07k 0:00:03 [ 638 /s] [    <=>                                               ]


In [ ]:
#Загрузим и соберем в один дата-сет
a_act = pd.read_csv('1_act.csv')
b_act = pd.read_csv('2_act.csv')
c_act = pd.read_csv('3_act.csv')
d_act = pd.read_csv('4_act.csv')
e_act = pd.read_csv('5_act.csv')
#У части спортсменов вес не был в данных, добавим руками
a_act['Athlete Weight'] = 90
b_act['Athlete Weight'] = 80
c_act['Athlete Weight'] = 85
allDf = pd.concat([a_act, b_act, c_act, d_act, e_act])

In [ ]:
#Выделим отдельно датасет для Визуализации
# allDf_visual = allDf[['Activity ID', 'Filename']].copy()

## Первичный анализ (Draft) можно пропустить

In [ ]:
#Инфа по всем столбцам
allDf.info()

In [ ]:
profile = allDf.profile_report()
profile.to_file(output_file="Pandas Profiling Report — Strava.html")

In [ ]:
#Не одно ли и то же?
allDf[['Commute','Activity Type']].loc[allDf['Commute'] == True] 
allDf[['Relative Effort','Relative Effort.1']]

In [ ]:
#Два столбца с расстоянием, один в КМ, второс Метрах - оставим последний, более информативный
allDf[['Distance','Distancet.1']]

In [ ]:
# Распределение по видам активности
RenameActType(a_act)
a_act['Activity Type'].value_counts()

In [ ]:
allDf['Activity Gear'].describe()

In [ ]:
allDf['Activity Gear'].value_counts()

In [ ]:
allDf['Athlete Weight'].value_counts()

In [ ]:
allDf['Activity Gear'].loc[allDf['Activity Gear'].isnull()] = 'Some Gear'

In [ ]:
allDf[['Commute','Commute.1']]

## Чистим данные

###Подготовка:
*   Признаки "*Activity Name*" и "*Activity Description*" описательные и заполняются самим пользователем и не несут смысловой нагрузки для анализа - удаляем
*   Признаки "*Distance*" и "*Distance.1*" один в км, второй метрах - оставим последний, более информативный
*   "*Activity Date*","*Activity ID*" - Первый таймСтамп - нам не нужен, второй - просто ID тренировки - удаляем
*  "*Commute.1*","*Commute*","*From Upload*" - технологические признаки связаные с загрузкой данных - не нужны
*   "*Filename*" - ссылка на доп файлы - не нужен
*   "*Activity Gear*" - тип спорт.оборудовния - описание, слишком очевидно для разделения по видам Активности








In [ ]:
#Объявим нужны функции и константы (признаки и типы активностей) для Baseline

#Функция мапинга английских и русских названий активностей
def RenameActType(df):
  df.loc[(df['Activity Type']=='Бег'), 'Activity Type'] = 'Run'
  df.loc[(df['Activity Type']=='Лыжи'), 'Activity Type'] = 'Nordic Ski'
  df.loc[(df['Activity Type']=='Велосипед'), 'Activity Type'] = 'Ride'
  df.loc[(df['Activity Type']=='Плавание'), 'Activity Type'] = 'Swim'
  df.loc[(df['Activity Type']=='Ходьба'), 'Activity Type'] = 'Walk'
  df.loc[(df['Activity Type']=='Роликовые коньки'), 'Activity Type'] = 'Inline Skate'
  df.loc[(df['Activity Type']=='Коньки'), 'Activity Type'] = 'Ice Skate'
  df.loc[(df['Activity Type']=='Горные лыжи'), 'Activity Type'] = 'Alpine Ski'

def DropNanColumns(df,col_to_drop):
  df.drop(col_to_drop,1,inplace = True)

top_activity = ['Run', 'Ride', 'Nordic Ski','Roller Ski','Walk','Inline Skate','Workout'] #ТОП-7 активностям, по колличеству записей
col_to_drop_m = ['Activity Name','Activity Description','Activity Date','Distance', 'Activity ID','Filename', 'From Upload', 'Commute', 'Commute.1'] #Уберем сразу часть признаков

### Baseline

In [ ]:
#Функция чистки
def CleanDataToLearning(allDf, col_to_drop_m = [], top_activity = []):
  RenameActType(allDf)
  col_to_drop = []
  allDf = allDf.loc[allDf['Activity Type'].isin(top_activity)]
  #col_to_drop = [feature for feature in allDf.columns if (allDf[feature].isnull().sum()/allDf[feature].size) >= 0.4] #Удаляем признаки, которые больше чем на 40% незаполнены
  col_to_drop += col_to_drop_m
  DropNanColumns(allDf,col_to_drop)
  return allDf

In [ ]:
def CutX_Y(df):
  X = allDf.copy()
  y = X['Activity Type']
  X.drop(['Activity Type'],1,inplace = True)
  return X, y

In [ ]:
def FeatureOptimaiser(df, col_to_drop_m, top_activity, correlation_threshold, missing_threshold, plotHMap):
  df = CleanDataToLearning(df, col_to_drop_m, top_activity)
  X_1, y_1  = CutX_Y(df)
  fs = FeatureSelector(data = X_1, labels = y_1)
  fs.identify_collinear(correlation_threshold)
  fs.identify_missing(missing_threshold)
  fs.identify_single_unique()
  if plotHMap:
    fs.plot_collinear()
  return fs.ops['collinear'], fs.ops['missing'], fs.ops['single_unique']

In [ ]:
# список признаков для удаления
collinear_features, missing_features, single_features = FeatureOptimaiser(allDf, col_to_drop_m, top_activity, 0.49, 0.6, False)

In [ ]:
fig = plt.figure(figsize=(8,8), dpi = 200)
corr_matrix = allDf.corr()
sns.heatmap(corr_matrix, annot=True, fmt=".2g", cmap="Oranges")

In [ ]:
plt.savefig("corr_mtx.png", format = 'png', dpi=200)

# Разделение датасета, Подготовка моделей и Обучение

## Разделение датасета на Трениновочную, Валидационную и Тестовые Выборки

###Вариант 1 (выделяем Тестовую, тренировочную и валидационную выборки из всего датасета)

In [ ]:
allDf = CleanDataToLearning(allDf, col_to_drop_m+collinear_features+missing_features+single_features, top_activity)

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
X, y  = CutX_Y(allDf)
categorical_feature = [feature for feature in X.columns if X[feature].dtypes == "object"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20) # выделим тестовую
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20) # разделм на тренировочную и валидационную

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


###Вариант 2 (выделяем Тестовую из данных одного спортсмена, а тренировочные и валидационные выборки из всех остальных)

In [ ]:
#Все кроме данных одного спортмена
allDf = pd.concat([a_act, e_act, b_act, c_act])
allDf = CleanDataToLearning(allDf, col_to_drop_m+collinear_features+missing_features+single_features, top_activity)
X_train, y_train = CutX_Y(allDf)

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [ ]:
#Данные одного на тест
d_act = CleanDataToLearning(d_act, col_to_drop_m+collinear_features+missing_features+single_features, top_activity)
X_test, y_test = CutX_Y(d_act)

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20) # разделм на тренировочную и валидационную

## Модели

In [ ]:
def ModelCatBoostClass(X_train, X_val, y_train, y_val, cat_features):
  train_dataset = Pool(data=X_train,
                     label=y_train,
                     cat_features=cat_features)

  eval_dataset = Pool(data=X_val,
                    label=y_val,
                    cat_features=cat_features)

  model_CBC = ctb.CatBoostClassifier(iterations=100,
                           learning_rate=1,
                           depth=4,
                           loss_function='MultiClass')
  model_CBC.fit(X_train, y_train)
  return model_CBC

## Обучение

In [ ]:
#Обучение модели
modelCBS = ModelCatBoostClass(X_train, X_val, y_train, y_val, categorical_feature)

0:	learn: 1.1947237	total: 50.6ms	remaining: 5s
1:	learn: 0.9356717	total: 53.8ms	remaining: 2.64s
2:	learn: 0.7474891	total: 57.2ms	remaining: 1.85s
3:	learn: 0.6447699	total: 60.4ms	remaining: 1.45s
4:	learn: 0.6084380	total: 63.4ms	remaining: 1.2s
5:	learn: 0.5684499	total: 66.6ms	remaining: 1.04s
6:	learn: 0.5413477	total: 69.6ms	remaining: 925ms
7:	learn: 0.5122031	total: 72.7ms	remaining: 836ms
8:	learn: 0.4900500	total: 76ms	remaining: 768ms
9:	learn: 0.4665941	total: 79ms	remaining: 711ms
10:	learn: 0.4401324	total: 82.1ms	remaining: 664ms
11:	learn: 0.4280878	total: 85.3ms	remaining: 626ms
12:	learn: 0.4228629	total: 88.3ms	remaining: 591ms
13:	learn: 0.4060985	total: 91.3ms	remaining: 561ms
14:	learn: 0.3789491	total: 94.5ms	remaining: 536ms
15:	learn: 0.3708691	total: 97.5ms	remaining: 512ms
16:	learn: 0.3485086	total: 100ms	remaining: 490ms
17:	learn: 0.3392041	total: 104ms	remaining: 472ms
18:	learn: 0.3295842	total: 107ms	remaining: 455ms
19:	learn: 0.3141119	total: 110ms

##Метрики

In [ ]:
#Метрики
expected_y  = y_test
predicted_y = modelCBS.predict(X_test)
print(metrics.classification_report(expected_y, predicted_y))
print(metrics.confusion_matrix(expected_y, predicted_y))

              precision    recall  f1-score   support

Inline Skate       0.82      0.38      0.51        24
  Nordic Ski       0.85      0.91      0.88        95
        Ride       0.90      0.91      0.90       130
  Roller Ski       0.79      0.93      0.86        29
         Run       0.89      0.92      0.91       185
        Walk       0.58      0.44      0.50        16
     Workout       0.67      0.50      0.57         8

    accuracy                           0.87       487
   macro avg       0.79      0.71      0.73       487
weighted avg       0.86      0.87      0.86       487

[[  9   4   3   2   4   2   0]
 [  0  86   3   2   4   0   0]
 [  1   2 118   1   7   0   1]
 [  0   2   0  27   0   0   0]
 [  1   4   7   2 171   0   0]
 [  0   2   0   0   6   7   1]
 [  0   1   0   0   0   3   4]]


In [ ]:
y_to_lab = expected_y.unique()
y_to_lab.sort(axis=0)
fig = plt.figure(figsize=(8,8), dpi = 200)
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20) 
sns.heatmap(metrics.confusion_matrix(y_test, modelCBS.predict(X_test)), annot=True, fmt="d", cmap="Greens", xticklabels=y_to_lab, yticklabels =y_to_lab)
plt.ylabel("Real value")
plt.xlabel("Predicted value")


Text(0.5, 115.44444444444443, 'Predicted value')

In [ ]:
plt.savefig("con_mtx.png", format = 'png', dpi=200)

# Визуализация - не актуально - решил отказаться - не имеет отношения к анализу

In [ ]:
allDf_visual

In [ ]:
def unpack_gz(in_file_name):
  gzip_file = gzip.GzipFile(in_file_name, 'rb')
  data = gzip_file.read()
  return data.decode('utf-8')

In [ ]:
gpx_file_1= unpack_gz('./activities/227157048.gpx.gz')
gpx_file_2= unpack_gz('./activities/227150014.gpx.gz')

In [ ]:
tcx_file_1= unpack_gz('./activities/325708135.tcx.gz')

In [ ]:
import tcxparser
tcx = tcxparser.TCXParser(tcx_file_1)

In [ ]:
import gpxpy
import gpxpy.gpx

def process_gpx_to_df(file_content):
  gpx = gpxpy.parse(file_content) 
 
  track = gpx.tracks[0]
  segment = track.segments[0]
  data = []
  segment_length = segment.length_3d()
  for point_idx, point in enumerate(segment.points):
    data.append([point.longitude, point.latitude,point.elevation,point.time, segment.get_speed(point_idx)])
    columns = ['Longitude', 'Latitude', 'Altitude', 'Time', 'Speed']
    gpx_df = pd.DataFrame(data, columns=columns)
 
  points = []
  for track in gpx.tracks:
    for segment in track.segments:
      for point in segment.points:
        points.append(tuple([point.latitude, point.longitude]))
  return gpx_df, points

In [ ]:
df_to_plt_1, points_to_plt_1 = process_gpx_to_df(gpx_file_1)
df_to_plt_2, points_to_plt_2 = process_gpx_to_df(gpx_file_2)

In [ ]:
df_to_plt_1

,Longitude,Latitude,Altitude,Time,Speed
0,75.190428,61.238244,39.0,2014-01-08 08:21:52+00:00,0.364655
1,75.190450,61.238213,39.0,2014-01-08 08:22:02+00:00,0.364655


In [ ]:
mymap = folium.Map( location=[ df_to_plt_1.Latitude.mean(), df_to_plt_1.Longitude.mean() ], zoom_start=10, tiles=None, width = '60%', height = '60%') #
folium.TileLayer('openstreetmap', name='OpenStreet Map').add_to(mymap)

In [ ]:
folium.PolyLine(points_to_plt_2, color='red', weight=4.5, opacity=.5).add_to(mymap)
folium.PolyLine(points_to_plt_1, color='blue', weight=4.5, opacity=.5).add_to(mymap)

In [ ]:
folium.Circle(location=[ df_to_plt_1.Latitude.mean(), df_to_plt_1.Longitude.mean()], radius = 1000, color = 'blue', fill = True, fill_opacity = 0.8, popup='Nordic Ski').add_to(mymap)
folium.Circle(location=[ df_to_plt_2.Latitude.mean(), df_to_plt_2.Longitude.mean()], radius = 1000, color = 'red', fill = True, fill_opacity = 0.8, popup='Some Act').add_to(mymap)

In [ ]:
mymap.png_enabled = False

In [ ]:
mymap